In [1]:
project      = 'hoshinolab_proteomics_age'
from bin import nimb_link
NIMB_HOME = nimb_link.link_with_nimb()
from setup.get_vars import Get_Vars, SetProject
from distribution import utilities as utils
from stats.db_processing import Table
from stats.preprocessing import Preprocess
from stats import preprocessing, varia
from stats.make_stats_grid import MakeGrid
from stats.stats_models import ANOVA_do
from stats.plotting import Make_Plot_Regression, Make_plot_group_difference
tab = Table()

all_vars = Get_Vars()
project_vars = all_vars.projects[project]

getvars      = Get_Vars()
nimb_stats   = getvars.stats_vars
projects     = getvars.projects
NIMB_tmp     = getvars.location_vars['local']['NIMB_PATHS']['NIMB_tmp']
fname_groups = projects[project]["fname_groups"]
nimb_stats   = SetProject(NIMB_tmp, nimb_stats, project, fname_groups).stats
project_vars = projects[project]
stats_paths  = nimb_stats['STATS_PATHS']
vars_glm     = project_vars['variables_for_glm']

def get_X_data_per_group_all_groups(group):
    # extract X_scaled values for the brain parameters
        predicted_target = project_vars["prediction_target"]
        if not predicted_target:
            predicted_target = project_vars["group_col"]
        if group == 'all':
                df_clin_group = df_user_stats
                df_X          = df_adjusted
                y_labeled     = preprocessing.label_y(df_user_stats, predicted_target)
                X_scaled      = preprocessing.scale_X(df_X)
        else:
                df_group      = tab.get_df_per_parameter(df_final_grid, project_vars['group_col'], group)
                df_clin_group = tab.rm_cols_from_df(df_group, cols_X)
                df_X          = tab.rm_cols_from_df(df_group, [i for i in df_group.columns.tolist() if i not in cols_X])
                y_labeled     = preprocessing.label_y(df_group, predicted_target)
                X_scaled      = preprocessing.scale_X(df_X)
        return df_X, y_labeled, X_scaled, df_clin_group

/home/ssp/Dropbox/github/nimb/nimb
        credentials are located at: /home/ssp/nimb
        credentials are located at: /home/ssp/nimb


In [2]:
df_user_stats, df_final_grid,\
            df_adjusted,\
            cols_X,\
            groups = MakeGrid(project_vars,
                                nimb_stats).grid()

    reading file: /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/stats_src.xlsx,
        sheet: 0
    reading file: /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/grid.csv,
        sheet: 0


In [3]:
col2rm = 'Unnamed: 0'
df_final_grid.drop(columns=[col2rm], inplace=True)

In [4]:
cols_with_zeros = list()
for col in df_final_grid.columns:
    n = df_final_grid[col].isin([0]).sum()
    if n>0:
        cols_with_zeros.append(col)
#        print(col, len(df_final_grid[col]), n)

In [32]:
df_final_grid.drop(columns=cols_with_zeros, inplace=True)
features = [i for i in features if i not in cols_with_zeros+[col2rm,]]
vars_glm = features#[i for i in vars_glm if i not in cols_with_zeros+[col2rm,]]

In [6]:
group = 'all'
df_X, y_labeled, X_scaled, df_clin_group = get_X_data_per_group_all_groups(group)
features = cols_X

PowerTransformer()


/home/ssp/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:3234: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [29]:
preproc = Preprocess(utils)
#(df_final_grid == 0).astype(int).sum(axis=0)
#(df_final_grid == 0).sum(axis=1)
#df_final_grid.isin([0]).sum()

In [33]:
sig_cols = ANOVA_do(df_final_grid,
                    vars_glm, features,
                    varia.get_dir(stats_paths['anova']),
                    p_thresh = 0.05, intercept_thresh = 0.05).sig_cols

    analysing 40 features for parameter: CON_00708398_1
    analysing 40 features for parameter: CON_LysC
    analysing 40 features for parameter: CON_Trypsin
    analysing 40 features for parameter: O43866
    analysing 40 features for parameter: P00450
    analysing 40 features for parameter: P00488
    analysing 40 features for parameter: P00739
    analysing 40 features for parameter: P01009
    analysing 40 features for parameter: P01011
    analysing 40 features for parameter: P01023
    analysing 40 features for parameter: P01024
    analysing 40 features for parameter: P01031
    analysing 40 features for parameter: P01597
    analysing 40 features for parameter: P01625
    analysing 40 features for parameter: P01764
    analysing 40 features for parameter: P01767
    analysing 40 features for parameter: P01833
    analysing 40 features for parameter: P01859
    analysing 40 features for parameter: P02042
    analysing 40 features for parameter: P02647
    analysing 40 features

In [34]:
Make_Plot_Regression(df_final_grid,
                    sig_cols, project_vars['group_col'],
                    varia.get_dir(stats_paths['simp_lin_reg_dir']))

creating folder:  /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/simple_lin_regression


In [25]:
Make_plot_group_difference(df_final_grid,
                            sig_cols, project_vars['group_col'], groups,
                            varia.get_dir(stats_paths['anova']))